# Getting familar with using the nba_api scraping API
## 5/13/25
This API scrapes live data off of NBA.com/stats, including game data, historical data, player stats
This data will be used this to develop an XGBoost algorithm to predict the winner of games for playoffs

In [2]:
import pandas as pd
import numpy as np
import json, difflib, time, requests
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
from nba_api.stats.static import teams, players

In [3]:
#Retry Wrapper
def retry(func, retries = 3):
    def retry_wrapper(*args, **kwargs):
        attempts = 0
        while attempts < retries:
            try:
                return func(*args, **kwargs)
            except requests.exceptions.RequestException as e:
                print(e)
                time.sleep(30)
                attemps +=1
    return retry_wrapper

In [7]:
def getSeasonScheduleFrame(seasons, seasonType):
    def getGameDate(matchup):
        return str(matchup).partition(' at')[0][:10]
    def getHomeTeam(matchup):
        return str(matchup).partition( 'at')[2]
    def getAwayTeam(matchup):
        return str(matchup).partition(' at')[0][10:]
    def getTeamIDFromNickname(nickname):
        return nickname
        # return str(teamLookup.loc[teamLookup['nickname'] == difflib.get_close_matches(nickname,teamLookup['nickname'],1)[0]].values[0][0])

    @retry
    def getRegularSeasonSchedule(season, teamID, seasonType):
        season = str(season) + "-" + str(season+1)[-2:]
        teamGames = cumestatsteamgames.CumeStatsTeamGames(league_id='00',season=season,
                                                          season_type_all_star=seasonType,
                                                          team_id=teamID).get_normalized_json()
        teamGames = pd.DataFrame(json.loads(teamGames)['CumeStatsTeamGames'])
        teamGames['SEASON']=season
        return teamGames
    
    teamLookup = pd.DataFrame(teams.get_teams())
    scheduleFrame = pd.DataFrame()
    for season in seasons:
        for id in teamLookup['id']:
            time.sleep(1)
            scheduleFrame = scheduleFrame.add(getRegularSeasonSchedule(season,id,seasonType))

    scheduleFrame['GAME_DATE'] = pd.to_datetime(scheduleFrame['MATCHUP'].map(getGameDate))
    scheduleFrame['HOME_TEAM_NICKNAME'] = scheduleFrame['MATCHUP'].map(getHomeTeam)
    scheduleFrame['HOME_TEAM_ID'] = scheduleFrame['MATCHUP'].map(getTeamIDFromNickname)
    scheduleFrame['AWAY_TEAM_NICKNAME'] = scheduleFrame['MATCHUP'].map(getAwayTeam)
    scheduleFrame['AWAY_TEAM_ID'] = scheduleFrame['MATCHUP'].map(getTeamIDFromNickname)
    scheduleFrame = scheduleFrame.drop_duplicates()
    scheduleFrame = scheduleFrame.reset_index(drop=True)

    return scheduleFrame

In [4]:
def getSingleGameMetrics(gameID, homeTeamID, awayTeamID, awayTeamNickname, seasonYear, gameDate):
    @retry
    def getGameStats(teamID, gameID, seasonYear):
        gameStats = cumestatsteam.CumeStatsTeam(game_ids = gameID, league_id="00",
                                                season=seasonYear, season_type_all_star="Regular Season",
                                                team_id=teamID).get_normalized_json()
        gameStats = pd.DataFrame(json.loads(gameStats)['TotalTeamStats'])
        return gameStats
    data = getGameStats(homeTeamID,gameID,seasonYear)
    data.at[1,'NICKNAME']=awayTeamNickname
    data.at[1,'TEAM_ID']=awayTeamID
    data.at[1,'OFFENSIVE_EFFICIENCY']=(data.at[1,'FG']+data.at[1,'AST'])/(data.at[1,'FGA']-data.at[1,'OFF_REB']+data.at[1,'AST']+data.at[1,'TOTAL_TURNOVERS'])
    data.at[1,'SCORING_MARGIN'] = data.at[1,'PTS']-data.at[0,'PTS']

    data.at[0,'OFFENSIVE_EFFICIENCY']=(data.at[0,'FG']+data.at[0,'AST'])/(data.at[0,'FGA']-data.at[0,'OFF_REB']+data.at[0,'AST']+data.at[0,'TOTAL_TURNOVERS'])
    data.at[1,'SCORING_MARGIN'] = data.at[0,'PTS']-data.at[1,'PTS']
    
    data['SEASON']=seasonYear
    data['GAME_DATE']=gameDate
    data['GAME_ID']=gameID

    return data

In [5]:
def getGameLogs(gameLogs,scheduleFrame):
    
    # Functions to prepare additional columns after gameLogs table loads
    def getHomeAwayFlag(gameDF):
        gameDF['HOME_FLAG'] = np.where((gameDF['W_HOME']==1) | (gameDF['L_HOME']==1),1,0)
        gameDF['AWAY_FLAG'] = np.where((gameDF['W_ROAD']==1) | (gameDF['L_ROAD']==1),1,0) 

    def getTotalWinPctg(gameDF):
        gameDF['TOTAL_GAMES_PLAYED'] = gameDF.groupby(['TEAM_ID','SEASON'])['GAME_DATE'].rank(ascending=True)
        gameDF['TOTAL_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W'].cumsum()
        gameDF['TOTAL_WIN_PCTG'] = gameDF['TOTAL_WINS']/gameDF['TOTAL_GAMES_PLAYED']
        return gameDF.drop(['TOTAL_GAMES_PLAYED','TOTAL_WINS'],axis=1)

    def getHomeWinPctg(gameDF):
        gameDF['HOME_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['HOME_FLAG'].cumsum()
        gameDF['HOME_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W_HOME'].cumsum()
        gameDF['HOME_WIN_PCTG'] = gameDF['HOME_WINS']/gameDF['HOME_GAMES_PLAYED']
        return gameDF.drop(['HOME_GAMES_PLAYED','HOME_WINS'],axis=1)

    def getAwayWinPctg(gameDF):
        gameDF['AWAY_GAMES_PLAYED'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['AWAY_FLAG'].cumsum()
        gameDF['AWAY_WINS'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['W_ROAD'].cumsum()
        gameDF['AWAY_WIN_PCTG'] = gameDF['AWAY_WINS']/gameDF['AWAY_GAMES_PLAYED']
        return gameDF.drop(['AWAY_GAMES_PLAYED','AWAY_WINS'],axis=1)

    def getRollingOE(gameDF):
        gameDF['ROLLING_OE'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['OFFENSIVE_EFFICIENCY'].transform(lambda x: x.rolling(3, 1).mean())

    def getRollingScoringMargin(gameDF):
        gameDF['ROLLING_SCORING_MARGIN'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['SCORING_MARGIN'].transform(lambda x: x.rolling(3, 1).mean())

    def getRestDays(gameDF):
        gameDF['LAST_GAME_DATE'] = gameDF.sort_values(by='GAME_DATE').groupby(['TEAM_ID','SEASON'])['GAME_DATE'].shift(1)
        gameDF['NUM_REST_DAYS'] = (gameDF['GAME_DATE'] - gameDF['LAST_GAME_DATE'])/np.timedelta64(1,'D') 
        return gameDF.drop('LAST_GAME_DATE',axis=1)
    
    start = time.perf_counter_ns()

    i = int(len(gameLogs)/2) #Can use a previously completed gameLog dataset

    while i<len(scheduleFrame):


        time.sleep(1)
        gameLogs =  gameLogs.append(getSingleGameMetrics(scheduleFrame.at[i,'GAME_ID'],scheduleFrame.at[i,'HOME_TEAM_ID'],
                         scheduleFrame.at[i,'AWAY_TEAM_ID'],scheduleFrame.at[i,'AWAY_TEAM_NICKNAME'],
                         scheduleFrame.at[i,'SEASON'],scheduleFrame.at[i,'GAME_DATE']))
        
        gameLogs = gameLogs.reset_index(drop=True)

        end = time.perf_counter_ns()

        #Output time it took to load x amount of records
        if i%100 == 0:
            mins = ((end-start)/1e9)/60
            print(i,str(mins) + ' minutes')

        i+=1
        
    # Get Table Level Aggregation Columns
    getHomeAwayFlag(gameLogs)
    gameLogs = getHomeWinPctg(gameLogs)
    gameLogs = getAwayWinPctg(gameLogs)
    gameLogs = getTotalWinPctg(gameLogs)
    getRollingScoringMargin(gameLogs)
    getRollingOE(gameLogs)
    gameLogs = getRestDays(gameLogs)

    return gameLogs.reset_index(drop=True)

In [8]:
#Get ScheduleFrame

seasons = [2022,2023,2024]
seasonType = 'Regular Season'

start = time.perf_counter_ns() # Track cell's runtime
scheduleFrame = getSeasonScheduleFrame(seasons,seasonType)
end = time.perf_counter_ns()

secs = (end-start)/1e9
mins = secs/60
print(mins)

1.6957100083333332


In [9]:
#Example Output of Single Game Metrics
getSingleGameMetrics(scheduleFrame.at[104,'GAME_ID'],scheduleFrame.at[104,'HOME_TEAM_ID'],
                     scheduleFrame.at[104,'AWAY_TEAM_ID'],scheduleFrame.at[104,'AWAY_TEAM_NICKNAME'],
                     scheduleFrame.at[104,'SEASON'],scheduleFrame.at[104,'GAME_DATE'])

KeyError: 104